<a href="https://colab.research.google.com/github/varekarprajwal/HPCS_LBP/blob/main/HPCS_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title CREATING THE REQUIRED ENVIROMENT
!sudo apt update
!sudo apt install libopencv-dev python3-opencv
!apt-get install -y mpich
!apt-get install -y openmpi-bin openmpi-doc openmpi-dev
!apt-get install -y g++
import numpy as np
import tensorflow as tf

In [ ]:
# @title SELECTING THE PATH
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Project/OPENCV/

In [124]:
# @title SEQUENTIAL CODE
%%writefile Sequential_OPENCV.cpp
#include <stdio.h>
#include <ctime>
#include <opencv4/opencv2/core.hpp>
#include <opencv4/opencv2/highgui.hpp>
#include <opencv4/opencv2/opencv.hpp>

using namespace cv;
using namespace std;

double getCurrentTime()
{
    struct timespec currentTime;
    clock_gettime(CLOCK_MONOTONIC, &currentTime);
    return (double)currentTime.tv_sec * 1000.0 + (double)currentTime.tv_nsec / 1000000.0;
}

int t_Xor[3][3] = {0};
int txt_img[2080][2080] = {0};

int wgt[3][3] = {
    {8, 4, 2},
    {16, 0, 1},
    {32, 64, 128}
};

int casecheck(int a, int b, int c, int d) {
   if (a == b && b == c && c == d && d == a)
        return 7;
    else if (a == b)
        return 1;
    else if (b == d)
        return 2;
    else if (c == d)
        return 3;
    else if (a == c)
        return 4;
    else if (a == d)
        return 5;
    else if (c == b)
        return 6;

    else
        return 0;
}

int Texton_weight(int fx, int fy, int lx, int ly) {
    int k = 0, l = 0;

    for (int i = fx; i < lx; i++) {
        for (int j = fy; j < ly; j++) {
            //printf("%d - %d\n", txt_img[fx + 1][fy + 1], txt_img[i][j]);
            if (txt_img[fx +1][fy +1] == txt_img[i][j])
                t_Xor[k][l] = 0;
            else {
                t_Xor[k][l] = 1;
            }
            l++;
        }
        l = 0;
        k++;
    }

    int xor_S = 0;
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            xor_S = xor_S + (t_Xor[i][j] * wgt[i][j]);
        }
    }
    return xor_S;
}

int main() {
  int i, j;

  Mat image = imread("sample_image/30x30.jpg");
  Mat hsv_image;
  cvtColor(image, hsv_image, COLOR_BGR2HSV);
  int m_r = hsv_image.rows / 2, m_c = hsv_image.cols / 2;
  int img[hsv_image.rows][hsv_image.cols] = {0};
  int Main_res[m_r][m_c] = {0};


  for (i = 0; i < hsv_image.rows; i++) {
    for (j = 0; j < hsv_image.cols; j++) {
      Vec3b hsv_pixel = hsv_image.at<Vec3b>(i, j);
      int hue = hsv_pixel[0];
      int saturation = hsv_pixel[1];
      int value = hsv_pixel[2];
      img[i][j] = value;
      printf("%3d ", value);
      }
    printf("\n");
    }

    int fx = 0;
    int fy = 0;
    int lx = hsv_image.rows;
    int ly = hsv_image.cols;

    int k = 0, l = 0;
  double startTime1 = getCurrentTime();
  for (i = fx; i < lx; i = i + 2) {
    for (j = fy; j < ly; j = j + 2) {
      int a, b, c, d;
      a = img[i][j], b = img[i][j + 1], c = img[i + 1][j], d = img[i + 1][j + 1];
      int rs = casecheck(a, b, c, d);
      txt_img[k][l] = rs;
      l++;
      }
      l = 0;
      k++;
    }
  double endTime1 = getCurrentTime();
  double totalTime1 = endTime1 - startTime1;

  printf("___________________\n");
  printf("\nTexton image\n \n");

  for (i = fx; i < m_r; i++) {
    for (j = fy; j < m_c; j++)
      printf("%2d  ", txt_img[i][j]);
      printf("\n");
    }

  printf("___________________\n");
  printf("\nTexton Weight image\n \n");


  double startTime2 = getCurrentTime();
  for (i = fx; i < m_r; i++) {
    for (j = fy; j < m_c; j++) {
      if (i == 0 || i == m_r - 1 || j == 0 || j == m_c - 1) {
        Main_res[i][j] = txt_img[i][j];
        }
      else{
        Main_res[i][j] = Texton_weight(i - 1, j - 1, i + 2, j + 2);
          }
      printf("%3d  ", Main_res[i][j]);
      }
    printf("\n");
    }
  double endTime2 = getCurrentTime();
  double totalTime2 = endTime2 - startTime2;

  printf("Elapsed time for texton : %f mili seconds\n", totalTime1);
  printf("Elapsed time for LTxXORp: %f mili seconds\n", totalTime2);
  printf("Elapsed time: %f mili seconds\n", totalTime1+totalTime2);
  return 0;
}

Overwriting Sequential_OPENCV.cpp


In [ ]:
# @title COMMAND RUN SEQUENTIAL CODE
!g++ Sequential_OPENCV.cpp -o app `pkg-config --cflags --libs opencv4` && ./app

In [ ]:
# @title COMMAND RUN PARALLEL MPI CODE
!mpic++ par3.cpp -o app `pkg-config --cflags --libs opencv4` && mpirun --allow-run-as-root -np 1 ./app

In [118]:
# @title CUDA CODE
%%writefile parallel_CUDA_OPENCV.cu

#include <iostream>
#include <stdio.h>
#include <opencv4/opencv2/core.hpp>
#include <opencv4/opencv2/highgui.hpp>
#include <opencv4/opencv2/opencv.hpp>
#include <cuda_runtime.h>
#include <ctime>

using namespace cv;
using namespace std;

double getCurrentTime()
{
    struct timespec currentTime;
    clock_gettime(CLOCK_MONOTONIC, &currentTime);
    return (double)currentTime.tv_sec * 1000.0 + (double)currentTime.tv_nsec / 1000000.0;
}



__device__ int getGlobalID_3D_3D(){
  int blockId = blockIdx.x + blockIdx.y * gridDim.x+ gridDim.x * gridDim.y * blockIdx.z;
  int threadId = blockId * (blockDim.x * blockDim.y * blockDim.z)+ (threadIdx.z * (blockDim.x * blockDim.y))+ (threadIdx.y * blockDim.x) + threadIdx.x;
  return threadId;
  }

__device__ int casecheck(int a, int b, int c, int d) {
   if (a == b && b == c && c == d && d == a)
        return 7;
    else if (a == b)
        return 1;
    else if (b == d)
        return 2;
    else if (c == d)
        return 3;
    else if (a == c)
        return 4;
    else if (a == d)
        return 5;
    else if (c == b)
        return 6;
    else
        return 0;
}


__global__ void calculate_TEXTON_CUDA(int* img_s, int* d_t_img, int rows, int cols)  {
  int x = getGlobalID_3D_3D();
  int a, b, c, d, r, i, q;

  q=x%(cols/2);
  r =x/(cols/2);
  i=((r*2)*cols)+(q*2);

  a = img_s[i], b = img_s[i + 1], c = img_s[cols+i], d = img_s[cols+i+1];
  //printf("x %d value %d %d %d %d \n",x,a,b,c,d);
  int rs = casecheck(a, b, c, d);
  d_t_img[x]=rs;
}

__global__ void calculate_LBP_CUDA(int* txt_img, int* Main_res, int rows, const int cols)  {
  int x = getGlobalID_3D_3D();
  int i =x;
  const int a=16,b=15,c=14,d=1,e=1,f=14,g=15,h=16;

  if ( i>cols && i<((rows-1)*cols)-1 ){
    if (txt_img[x] == txt_img[x-a])//6
      txt_img[x-a] = 0;
    else
      txt_img[x-a] = 1;

    if (txt_img[x] == txt_img[x-b]) //5
      txt_img[x-b] = 0;
    else
      txt_img[x-b] = 1;

    if (txt_img[x] == txt_img[x-c]) //4
      txt_img[x-c] = 0;
    else
      txt_img[x-c] = 1;

    if (txt_img[x] == txt_img[x-d]) //1
      txt_img[x-d] = 0;
    else
      txt_img[x-d] = 1;

    if (txt_img[x] == txt_img[x+e])//1
      txt_img[x+e] = 0;
    else
      txt_img[x+e] = 1;

    if (txt_img[x] == txt_img[x+f]) //4
      txt_img[x+f] = 0;
    else
      txt_img[x+f] = 1;

    if (txt_img[x] == txt_img[x+g]) //5
      txt_img[x+g] = 0;
    else
      txt_img[x+g] = 1;

    if (txt_img[x] == txt_img[x+h]) //6
      txt_img[x+h] = 0;
    else
      txt_img[x+h] = 1;

  int xor_S = 0;
  const int wgt[9] = {8, 4, 2,16, 0, 1,32, 64, 128};
  xor_S =(txt_img[x-a] * wgt[0])+(txt_img[x-b] * wgt[1])+(txt_img[x-c] * wgt[2])+(txt_img[x-d] * wgt[3])+(txt_img[x+e] * wgt[5])+(txt_img[x+f] * wgt[6])+(txt_img[x+g] * wgt[7])+(txt_img[x+h] * wgt[8]);
  Main_res[x]=xor_S;
  }

}

int main() {
  int i, j;
  int t_m_r,t_m_c;
  int i_m_r,i_m_c;
  int *h_img_r = new int[1000*1000];
  Mat image = imread("sample_image/30x30.jpg");

  if (image.empty()) {
    cerr << "Error: Couldn't load input image." << endl;
    return -1;
    }

  Mat hsv_image;
  cvtColor(image, hsv_image, COLOR_BGR2HSV);
  i_m_r = hsv_image.rows ;
  i_m_c = hsv_image.cols ;
  t_m_r = hsv_image.rows / 2;
  t_m_c = hsv_image.cols / 2;


  int k=0;
  for (i = 0; i < hsv_image.rows; i++) {
    for (j = 0; j < hsv_image.cols; j++) {
      Vec3b hsv_pixel = hsv_image.at<Vec3b>(i, j);
      int value = hsv_pixel[2];
      h_img_r[k]=value;
      printf("%3d ", h_img_r[k]);
      k++;
      }
    printf("\n");
  }
  printf("____________________\n");
  printf("\nTexton image\n \n");

  int* d_img;
  int* d_t_img;
  int imageSize = i_m_r * i_m_c * sizeof(int);
  int t_imageSize = t_m_r * t_m_c * sizeof(int);

  cudaMalloc((void **)&d_img, imageSize);
  cudaMalloc((void **)&d_t_img, imageSize);


  cudaMemcpy(d_img, h_img_r, imageSize, cudaMemcpyHostToDevice);


  dim3 threadsPerBlock(16, 16);
  dim3 numBlocks1((i_m_c + threadsPerBlock.x - 1) / threadsPerBlock.x, (i_m_r + threadsPerBlock.y - 1) / threadsPerBlock.y);
  dim3 numBlocks2((t_m_c + threadsPerBlock.x - 1) / threadsPerBlock.x, (t_m_r + threadsPerBlock.y - 1) / threadsPerBlock.y);

  double startTime1 = getCurrentTime();

  calculate_TEXTON_CUDA<<<numBlocks1, threadsPerBlock>>>(d_img, d_t_img, i_m_r , i_m_c);

  double endTime1 = getCurrentTime();
  int *h_T_img = new int[t_m_r *t_m_r ];
  cudaMemcpy(h_T_img, d_t_img, t_imageSize, cudaMemcpyDeviceToHost);

  for (i = 0; i < t_m_r; i++) {
    for (j = 0; j < t_m_c; j++) {
      printf("%4d", h_T_img[i*t_m_c+j]);
        }
    printf("\n");
  }

  printf("___________________\n");
  printf("\nTexton Weight image\n \n");


  int *h_Txt_img = new int[t_m_r *t_m_r ];
  cudaMalloc((void **)&h_Txt_img, imageSize);

  int *dr_Txt_img = new int[t_m_r *t_m_r ];
  cudaMalloc((void **)&dr_Txt_img, imageSize);

  cudaMemcpy(dr_Txt_img, h_T_img, imageSize, cudaMemcpyHostToDevice);
  cudaMemcpy(h_Txt_img, h_T_img, imageSize, cudaMemcpyHostToDevice);

  double startTime2 = getCurrentTime();

  calculate_LBP_CUDA<<<numBlocks2, threadsPerBlock>>>(dr_Txt_img, h_Txt_img, t_m_r , t_m_c);

  double endTime2 = getCurrentTime();

  int *MM_T_img = new int[t_m_r *t_m_r ];
  cudaMemcpy(MM_T_img, h_Txt_img, t_imageSize, cudaMemcpyDeviceToHost);

  for (i = 0; i < t_m_r; i++) {
    for (j = 0; j < t_m_c; j++) {
      printf("%4d", MM_T_img[i*t_m_c+j]);
        }
    printf("\n");
  }
  double totalTime1 = endTime1 - startTime1;
  printf("Time taken by Parallel to calculate Texton code: %.5f milliseconds\n", totalTime1);
  double totalTime2 = endTime2 - startTime2;
  printf("Time taken by Parallel to calculate LTxXORp code: %.5f milliseconds\n", totalTime2);
  printf("Elapsed time: %f mili seconds\n", totalTime1+totalTime2);

  return 0;
}


Overwriting parallel_CUDA_OPENCV.cu


In [ ]:
# @title COMMAND RUN PARALLEL CUDA CODE
!nvcc -o parallel parallel_CUDA_OPENCV.cu `pkg-config --cflags --libs opencv4` && ./parallel